In [1]:
!wget https://raw.githubusercontent.com/Rstam59/TaskDataRepoForStudents/main/housing.csv

--2025-09-13 18:24:34--  https://raw.githubusercontent.com/Rstam59/TaskDataRepoForStudents/main/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘housing.csv’

housing.csv         100%[===================>]   1.36M  --.-KB/s    in 0.05s   

2025-09-13 18:24:35 (26.0 MB/s) - ‘housing.csv’ saved [1423529/1423529]



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
housing=pd.read_csv("/content/housing.csv")
housing

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [4]:
X=housing.drop('median_house_value',axis=1)
y=housing ['median_house_value'].copy()

In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size =0.2)
X_train.shape,X_test.shape,y_train.shape, y_test.shape

((16512, 9), (4128, 9), (16512,), (4128,))

In [6]:
housing['rooms_per_house']=housing['total_rooms'] / housing['households']
housing['bedrooms_ration']=housing['total_bedrooms']/ housing['total_rooms']
housing['people_per_house']=housing['population']/ housing['households']

In [7]:
housing= housing.drop(housing[housing['median_house_value']>=500000].index)
housing=housing.drop(housing[housing['housing_median_age']>=50].index)

In [8]:
housing['total_bedrooms']=np.log(housing['total_bedrooms'])
housing['total_rooms'] = np.log (housing['total_rooms'])
housing['population'] = np.log (housing['population'])
housing['households']= np.log (housing['households'])

In [9]:
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,MinMaxScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer

cat_features=X_train.select_dtypes(exclude=[np.number]).columns
num_features=X_train.select_dtypes(include=[np.number]).columns
cat_features,num_features

(Index(['ocean_proximity'], dtype='object'),
 Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
        'total_bedrooms', 'population', 'households', 'median_income'],
       dtype='object'))

In [10]:
cat_pipeline=Pipeline([('imputer',SimpleImputer(strategy='most_frequent')),
                      ('encoder', OneHotEncoder(handle_unknown='ignore'))
                      ])
num_pipeline=make_pipeline(
    SimpleImputer(strategy='median'),StandardScaler()
    )



transformer = ColumnTransformer([('cat', cat_pipeline, cat_features),
                                 ('num', num_pipeline, num_features)])

In [11]:
X_train_transformed=transformer.fit_transform(X_train)

X_test_transformed=transformer.transform(X_test)

feature_names =transformer.get_feature_names_out()

X_train_transformed_df=pd.DataFrame(X_train_transformed,columns=feature_names)
X_test_transformed_df=pd.DataFrame(X_test_transformed,columns=feature_names)


In [12]:
from sklearn.linear_model import LinearRegression
lin_reg=LinearRegression()
lin_reg.fit(X_train_transformed_df,y_train)


LinearRegression()

In [13]:
lin_reg.score(X_test_transformed_df,y_test)

0.6302158091664074

In [14]:
from sklearn.preprocessing import PolynomialFeatures

poly_features=PolynomialFeatures(degree=2,include_bias=False)
X_poly=poly_features.fit_transform(X_train_transformed_df)


In [15]:
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y_train)

LinearRegression()

In [16]:
lin_reg.score(X_poly, y_train)

0.7162494514864548

In [17]:
from sklearn.linear_model import Ridge #l2

ridge_reg =Ridge(alpha=0.1,solver="cholesky")
ridge_reg.fit(X_poly,y_train)


Ridge(alpha=0.1, solver='cholesky')

In [18]:
ridge_reg.score(X_poly,y_train)

0.7162442330940528

In [19]:
from sklearn.linear_model import Lasso #l1

lasso_reg=Lasso(alpha=0.1)
lasso_reg.fit(X_poly,y_train)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.129e+13, tolerance: 2.205e+10
  model = cd_fast.enet_coordinate_descent(


Lasso(alpha=0.1)

In [20]:
lasso_reg.score(X_poly,y_train)

0.7159381121073015

In [21]:
from sklearn.linear_model import ElasticNet

elastic_net=ElasticNet(alpha=0.1,l1_ratio=0.5)
elastic_net.fit(X_poly,y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.914e+13, tolerance: 2.205e+10
  model = cd_fast.enet_coordinate_descent(


ElasticNet(alpha=0.1)

In [22]:
elastic_net.score(X_poly,y_train)

0.683349917375084